# Exploration, testing, visualization

## Step 2: Load and visualize point clouds

This notebook demonstrates how to load and visualize point clouds using Open3D.


In [ ]:
import sys
from pathlib import Path

# Add backend to path
sys.path.append(str(Path().resolve().parent))

from backend.data.datasets import load_point_cloud, get_point_cloud_info
from backend.data.preprocessing import normalize_point_cloud
from backend.visualization.visualize_o3d import show_point_cloud
import open3d as o3d
import numpy as np


### Example 1: Create and visualize a synthetic point cloud


In [ ]:
# Create a simple sphere point cloud for testing
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd = mesh.sample_points_uniformly(number_of_points=2000)

# Display info
info = get_point_cloud_info(pcd)
print(f"Point cloud info: {info}")

# Normalize
pcd_normalized = normalize_point_cloud(pcd)

# Visualize
show_point_cloud(pcd_normalized, window_name="Synthetic Sphere")


### Example 2: Load a point cloud from file

To load a .pcd or .ply file, uncomment and modify the path below:


In [ ]:
# Uncomment and set your file path:
# file_path = "path/to/your/pointcloud.pcd"  # or .ply
# pcd = load_point_cloud(file_path)
# pcd_normalized = normalize_point_cloud(pcd)
# show_point_cloud(pcd_normalized)


## Step 3: Preprocessing - Downsampling, Normalization, Partial Views

This section demonstrates preprocessing operations on point clouds.


In [ ]:
from backend.data.preprocessing import (
    downsample_voxel,
    normalize_point_cloud,
    create_partial_point_cloud,
    add_gaussian_noise,
    mask_points_by_angle
)


### Example 1: Downsampling with Voxel Grid


In [ ]:
# Create a dense point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=50)
pcd_original = mesh.sample_points_uniformly(number_of_points=10000)

print(f"Original: {len(pcd_original.points)} points")

# Downsample
pcd_downsampled = downsample_voxel(pcd_original, voxel_size=0.1)
print(f"Downsampled: {len(pcd_downsampled.points)} points")

# Visualize comparison
from backend.visualization.visualize_o3d import show_point_clouds
show_point_clouds([pcd_original, pcd_downsampled], window_name="Original vs Downsampled")


### Example 2: Normalization Methods


In [ ]:
# Create a point cloud with arbitrary scale and position
mesh = o3d.geometry.TriangleMesh.create_box(width=5, height=3, depth=2)
pcd_original = mesh.sample_points_uniformly(number_of_points=2000)

# Apply different normalization methods
pcd_unit_sphere = normalize_point_cloud(pcd_original, method='unit_sphere')
pcd_centered = normalize_point_cloud(pcd_original, method='centered')
pcd_zero_one = normalize_point_cloud(pcd_original, method='zero_one')

print("Normalization methods applied:")
print(f"  Original: {np.asarray(pcd_original.points)[0]}")
print(f"  Unit sphere: {np.asarray(pcd_unit_sphere.points)[0]}")
print(f"  Centered: {np.asarray(pcd_centered.points)[0]}")
print(f"  Zero-one: {np.asarray(pcd_zero_one.points)[0]}")

# Visualize
show_point_clouds([pcd_original, pcd_unit_sphere], window_name="Original vs Normalized")


### Example 3: Create Partial Point Cloud (Mask Side)


In [ ]:
# Create a complete point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd_complete = mesh.sample_points_uniformly(number_of_points=3000)

# Create partial view (mask one side)
pcd_partial = create_partial_point_cloud(
    pcd_complete,
    method='mask_side',
    direction=[1, 0, 0],  # Mask points on +X side
    mask_ratio=0.5
)

print(f"Complete: {len(pcd_complete.points)} points")
print(f"Partial: {len(pcd_partial.points)} points")

# Visualize
show_point_clouds([pcd_complete, pcd_partial], window_name="Complete vs Partial")


### Example 4: Simulate Sensor Field of View (Angle > 45°)


In [ ]:
# Create a point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd_complete = mesh.sample_points_uniformly(number_of_points=3000)

# Method 1: Using create_partial_point_cloud with angle masking
pcd_angle_masked = create_partial_point_cloud(
    pcd_complete,
    method='mask_angle',
    direction=[1, 0, 0],  # Sensor looking along +X
    max_angle=45.0  # Keep only points within 45° field of view
)

# Method 2: Using mask_points_by_angle (same result)
pcd_angle_masked2 = mask_points_by_angle(
    pcd_complete,
    view_direction=[1, 0, 0],
    max_angle=45.0
)

print(f"Complete: {len(pcd_complete.points)} points")
print(f"Angle masked (method 1): {len(pcd_angle_masked.points)} points")
print(f"Angle masked (method 2): {len(pcd_angle_masked2.points)} points")

# Visualize
show_point_clouds([pcd_complete, pcd_angle_masked], window_name="Complete vs Angle Masked")


### Example 5: Add Gaussian Noise


In [ ]:
# Create a clean point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd_clean = mesh.sample_points_uniformly(number_of_points=2000)

# Add Gaussian noise
pcd_noisy = add_gaussian_noise(pcd_clean, std=0.02, relative=True)

print(f"Clean: {len(pcd_clean.points)} points")
print(f"Noisy: {len(pcd_noisy.points)} points")

# Visualize
show_point_clouds([pcd_clean, pcd_noisy], window_name="Clean vs Noisy")


### Example 6: Complete Preprocessing Pipeline


In [ ]:
# Complete preprocessing pipeline
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=40)
pcd_original = mesh.sample_points_uniformly(number_of_points=5000)

print("Original:", len(pcd_original.points), "points")

# Step 1: Downsample
pcd = downsample_voxel(pcd_original, voxel_size=0.05)
print("After downsampling:", len(pcd.points), "points")

# Step 2: Create partial view (simulate sensor)
pcd = create_partial_point_cloud(pcd, method='mask_angle', direction=[1, 0, 0], max_angle=45.0)
print("After angle masking:", len(pcd.points), "points")

# Step 3: Add noise (simulate sensor noise)
pcd = add_gaussian_noise(pcd, std=0.01, relative=True)
print("After noise:", len(pcd.points), "points")

# Step 4: Normalize
pcd = normalize_point_cloud(pcd, method='unit_sphere')
print("After normalization:", len(pcd.points), "points")

# Visualize before and after
show_point_clouds([pcd_original, pcd], window_name="Before vs After Preprocessing")


## Step 4: Classical Methods - Normal Estimation + Surface Reconstruction

This section demonstrates classical baseline methods:
- PCA-based normal estimation (k-NN + local PCA)
- Poisson Surface Reconstruction


In [ ]:
from backend.geometry.normals import estimate_normals_pca, get_k_nearest_neighbors, compute_local_pca
from backend.geometry.reconstruction import (
    poisson_surface_reconstruction,
    mesh_to_point_cloud,
    reconstruct_and_convert
)


### Example 1: Estimate Normals using PCA (k-NN + Local PCA)


In [ ]:
# Create a point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd = mesh.sample_points_uniformly(number_of_points=2000)
pcd = normalize_point_cloud(pcd, method='unit_sphere')

print(f"Original point cloud: {len(pcd.points)} points")
print(f"Has normals: {pcd.has_normals()}")

# Estimate normals using PCA
pcd_with_normals = estimate_normals_pca(pcd, k=30, orient_normals=True)

print(f"\nAfter PCA normal estimation:")
print(f"Has normals: {pcd_with_normals.has_normals()}")
if pcd_with_normals.has_normals():
    normals = np.asarray(pcd_with_normals.normals)
    print(f"Normal sample: {normals[0]}")
    print(f"Normal magnitude: {np.linalg.norm(normals[0]):.4f}")

# Visualize with normals
show_point_cloud(pcd_with_normals, window_name="Point Cloud with PCA Normals")


### Example 2: Poisson Surface Reconstruction


In [ ]:
# Ensure we have normals
if not pcd_with_normals.has_normals():
    pcd_with_normals = estimate_normals_pca(pcd, k=30)

# Perform Poisson reconstruction
print("Performing Poisson Surface Reconstruction...")
mesh = poisson_surface_reconstruction(
    pcd_with_normals,
    depth=9,  # Higher = more detail but slower
    scale=1.1
)

print(f"Reconstructed mesh:")
print(f"  Vertices: {len(mesh.vertices)}")
print(f"  Triangles: {len(mesh.triangles)}")

# Convert mesh back to point cloud for comparison
pcd_reconstructed = mesh_to_point_cloud(mesh, number_of_points=len(pcd.points))

print(f"  Reconstructed point cloud: {len(pcd_reconstructed.points)} points")

# Visualize comparison
show_point_clouds(
    [pcd, pcd_reconstructed],
    window_name="Original vs Poisson Reconstructed"
)


### Example 3: Complete Pipeline (Load → Normals → Reconstruct → Visualize)


In [ ]:
# Complete pipeline demonstration
print("=" * 60)
print("Complete Pipeline: Normal Estimation + Poisson Reconstruction")
print("=" * 60)

# Step 1: Load or create point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd_original = mesh.sample_points_uniformly(number_of_points=2000)
pcd_original = normalize_point_cloud(pcd_original, method='unit_sphere')

print(f"\n1. Original point cloud: {len(pcd_original.points)} points")

# Step 2: Estimate normals using PCA
pcd_with_normals = estimate_normals_pca(pcd_original, k=30, orient_normals=True)
print(f"2. Normals estimated: {pcd_with_normals.has_normals()}")

# Step 3: Poisson reconstruction
mesh_reconstructed = poisson_surface_reconstruction(
    pcd_with_normals,
    depth=9,
    scale=1.1
)
print(f"3. Mesh reconstructed: {len(mesh_reconstructed.vertices)} vertices")

# Step 4: Convert to point cloud for comparison
pcd_reconstructed = mesh_to_point_cloud(mesh_reconstructed, number_of_points=len(pcd_original.points))
print(f"4. Converted to point cloud: {len(pcd_reconstructed.points)} points")

# Step 5: Visualize
print("\n5. Visualizing results...")
show_point_clouds(
    [pcd_original, pcd_reconstructed],
    window_name="Complete Pipeline: Original vs Reconstructed"
)

print("\nPipeline completed!")


## Step 5: Deep Learning Model - Sparse UNet

This section demonstrates the Sparse UNet model for point cloud completion.
Note: Requires MinkowskiEngine (Linux/WSL2 environment).


In [ ]:
try:
    from backend.models.sparse_unet import SparseUNet, create_sparse_tensor, sparse_tensor_to_point_cloud
    from backend.models.losses import ChamferLoss, FScoreLoss, CombinedLoss
    import MinkowskiEngine as ME
    import torch
    MINKOWSKI_AVAILABLE = True
    print("MinkowskiEngine is available!")
except ImportError as e:
    print(f"MinkowskiEngine not available: {e}")
    print("This model requires MinkowskiEngine (Linux/WSL2 environment).")
    MINKOWSKI_AVAILABLE = False


### Example 1: Create and Test Sparse UNet Model


In [ ]:
if MINKOWSKI_AVAILABLE:
    # Create model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")
    
    model = SparseUNet(in_channels=3, out_channels=3, base_channels=32)
    model = model.to(device)
    model.eval()
    
    # Count parameters
    num_params = sum(p.numel() for p in model.parameters())
    print(f"Model parameters: {num_params:,}")
    
    # Create dummy input
    num_points = 1000
    coordinates = torch.randint(0, 100, (num_points, 3), dtype=torch.long, device=device)
    batch_indices = torch.zeros(num_points, 1, dtype=torch.long, device=device)
    coordinates_with_batch = torch.cat([batch_indices, coordinates], dim=1)
    features = torch.randn(num_points, 3, device=device)
    
    input_sparse = ME.SparseTensor(
        features=features,
        coordinates=coordinates_with_batch,
        device=device
    )
    
    print(f"\nInput: {len(input_sparse.F)} points")
    
    # Forward pass
    with torch.no_grad():
        output_sparse = model(input_sparse)
    
    print(f"Output: {len(output_sparse.F)} points")
    print("✓ Forward pass successful!")
else:
    print("Skipping - MinkowskiEngine not available")


In [ ]:
if MINKOWSKI_AVAILABLE:
    # Create dummy predictions and targets
    pred_points = torch.randn(500, 3, device=device)
    target_points = torch.randn(800, 3, device=device)
    
    # Chamfer Distance
    chamfer_loss = ChamferLoss()
    chamfer_val = chamfer_loss(pred_points, target_points)
    print(f"Chamfer Distance: {chamfer_val.item():.6f}")
    
    # F-score
    fscore_loss = FScoreLoss(threshold=0.1)
    fscore_val = fscore_loss(pred_points, target_points)
    print(f"F-score Loss (1 - F): {fscore_val.item():.6f}")
    
    # Combined loss
    combined_loss = CombinedLoss(
        chamfer_weight=1.0,
        fscore_weight=0.5,
        normal_weight=0.0
    )
    combined_val = combined_loss(pred_points, target_points)
    print(f"Combined Loss: {combined_val.item():.6f}")
else:
    print("Skipping - MinkowskiEngine not available")
